In [1]:
#se importan los paquetes necesarios
import matplotlib.pyplot as plt
import numpy as np
import mpmath 
from ipywidgets import interact
from sympy.solvers import solve
from sympy.core.symbol import symbols

In [2]:
#función para definir los parámetros que se introducirán al modelo
def param_4sp(r1,r2,r3,r4,b11,b12,b13,b14,b21,b22,b23,b24,b31,b32,b33,b34,b41,b42,b43,b44,a1,a2,a3,a4,c1,c2,c3,c4):
    R=np.zeros((4,))
    C=np.zeros((4,))
    A=np.zeros((4,))
    B=np.zeros((4,4))
    R[0]=r1
    R[1]=r2
    R[2]=r3
    R[3]=r4
    B[0][0]=b11
    B[0][1]=b12
    B[0][2]=b13
    B[0][3]=b14
    B[1][0]=b21
    B[1][1]=b22
    B[1][2]=b23
    B[1][3]=b24
    B[2][0]=b31
    B[2][1]=b32
    B[2][2]=b33
    B[2][3]=b34
    B[3][0]=b41
    B[3][1]=b42
    B[3][2]=b43
    B[3][3]=b44
    A[0]=a1
    A[1]=a2
    A[2]=a3
    A[3]=a4
    C[0]=c1
    C[1]=c2
    C[2]=c3
    C[3]=c4
    return R,B,A,C

In [3]:
#función utilizada para encontrar las soluciones estacionarias del sistema de ecuaciones diferenciales
def solut_4sp(params):
    r,b,a,c = params  #se introducen los parámetros
    print(a)
    x1, x2, x3, x4= symbols('x1, x2, x3, x4', complex=True)
    
    #se definen las ecuaciones a resolver
    dx1=x1*(r[0] + (b[0][0]-a[0])*x1 + b[0][1]*x2 + b[0][2]*x3 + b[0][3]*x4- c[0]*x1*(b[0][0]*x1 + b[0][1]*x2 + b[0][2]*x3 + b[0][3]*x4))
    dx2=x2*(r[1] + (b[1][1]-a[1])*x2 + b[1][0]*x1 + b[1][2]*x3 + b[1][3]*x4- c[1]*x2*(b[1][0]*x1 + b[1][1]*x2 + b[1][2]*x3 + b[1][3]*x4))
    dx3=x3*(r[2] + (b[2][2]-a[2])*x3 + b[2][0]*x1 + b[2][1]*x2 + b[2][3]*x4- c[2]*x3*(b[2][0]*x1 + b[2][1]*x2 + b[2][2]*x3 + b[2][3]*x4))
    dx4=x4*(r[3] + (b[3][3]-a[3])*x4 + b[3][0]*x1 + b[3][1]*x2 + b[3][2]*x3- c[3]*x4*(b[3][0]*x1 + b[3][1]*x2 + b[3][2]*x3 + b[3][3]*x4))
    
    sols = solve([dx1, dx2, dx3, dx4],[x1, x2, x3, x4])  #se resuelve el sistema
    
    sol_compl = np.array((4,),dtype=complex)

    return sols  #la función devuelve las soluciones estacionarias dle sistema

In [5]:
r,b,a,c = param_4sp(0.15,0.25,0.15,0.15, 0.04,-0.18,0.01,-0.25, 0.12,-0.82,0.19,0.01, 0.000,0.36,-0.29,0.01, 0.01,0.01,0.01,0.01, 0.00075,0.00075,0.00075,0.00075, 0.005,0.005,0.005,0.005)
params=(r,b,a,c)  #definición manueal de los parámetros

sols=solut_4sp(params)  

[0.00075 0.00075 0.00075 0.00075]
<class 'sympy.core.mul.Mul'>


In [7]:
#construcción de la matriz Jacobiana para encontrar los autovalores
def Jacobian_4sp(X, params):
    r,b,a,c = params

    x1=X[0]
    x2=X[1]
    x3=X[2]
    x4=X[3]
    
    J11 = r[0] + b[0][1]*x2 + b[0][2]*x3 + b[0][3]*x4 + 2.*(b[0][0]-a[0])*x1 - 2.*c[0]*b[0][1]*x1*x2 - 2.*c[0]*b[0][2]*x1*x3 - 2.*c[0]*b[0][3]*x1*x4 - 3.*c[0]*b[0][0]*x1*x1
    J12 = b[0][1]*x1*(1. - c[0]*x1)
    J13 = b[0][2]*x1*(1. - c[0]*x1)
    J14 = b[0][3]*x1*(1. - c[0]*x1)
    
    J21 = b[1][0]*x2*(1. - c[1]*x2)
    J22 = r[1] + b[1][0]*x1 + b[1][2]*x3 + b[1][3]*x4 + 2.*(b[1][1]-a[1])*x2 - 2.*c[1]*b[1][0]*x2*x1 - 2.*c[1]*b[1][2]*x2*x3 - 2.*c[1]*b[1][3]*x2*x4 - 3.*c[1]*b[1][1]*x2*x2
    J23 = b[1][2]*x2*(1. - c[1]*x2)
    J24 = b[1][3]*x2*(1. - c[1]*x2)

    J31 = b[2][0]*x3*(1. - c[2]*x3)
    J32 = b[2][0]*x3*(1. - c[2]*x3)
    J33 = r[2]+ b[2][0]*x1 + b[2][1]*x2 + b[2][3]*x4 + 2.*(b[2][2]-a[2])*x3 - 2.*c[2]*b[2][0]*x3*x1 - 2.*c[2]*b[2][1]*x3*x2 - 2.*c[2]*b[2][3]*x3*x4 - 3.*c[2]*b[2][2]*x3*x3
    J34 = b[2][3]*x3*(1. - c[2]*x3)

    J41 = b[3][0]*x4*(1. - c[3]*x4)
    J42 = b[3][1]*x4*(1. - c[3]*x4)
    J43 = b[3][2]*x4*(1. - c[3]*x4)
    J44 = r[3] + b[3][0]*x1 + b[3][1]*x2 + b[3][2]*x3 + 2.*(b[3][3]-a[3])*x4 - 2.*c[3]*b[3][0]*x4*x1 - 2.*c[3]*b[3][1]*x4*x2 - 2.*c[3]*b[3][2]*x4*x3 - 3.*c[3]*b[3][3]*x4*x4

    """ Return the Jacobian matrix evaluated in X. """
    return np.array([[ J11, J12, J13, J14],
                  [J21, J22, J23, J24],
                  [J31, J32, J33, J34],
                  [J41, J42, J43, J44]],  dtype=complex)

In [8]:
Xss=[] #lista de soluciones estables
Lss=[] #lista de autovalores de dichas soluciones
X_finite=[] #lista de soluciones positivas
for s in sols:
    if s[0]>0 and s[1]>0 and s[2]>0 and s[3]>0:  #se comprueba que la solución tenga todos sus valores positivos
        print('Xsfinite:', s)
        X_f = ([s[0], s[1], s[2], s[3]])
        X_finite.append(X_f)
    
        print('X:', X_f)
        A_f1 = Jacobian_4sp(X_f,params)         #se define la matriz Jacobiana           
                                                
        lambda1, lambda2, lambda3, lambda4 = np.linalg.eigvals(A_f1) #se calculan los autovalores
        print('l1=',lambda1,'; l2=', lambda2, '; l3=',lambda3, '; l4=', lambda4)
        if(lambda1.real<0 and lambda2.real<0 and lambda3.real<0 and lambda4.real<0): #se comprueba que todos tengan su parte real menor que 0
            print('STABLE\n')
            Xss.append([s[0], s[1], s[2], s[3]])
            Lss.append([lambda1, lambda2, lambda3, lambda4])

Xsfinite: (200.000000000000, 47.4026477164719, 66.2584592342409, 200.000000000000)
X: [200.000000000000, 47.4026477164719, 66.2584592342409, 200.000000000000]
l1= (-29.62636211132484+0j) ; l2= (49.719891996622536+0j) ; l3= (-12.849187218791293+0j) ; l4= (-5.286611069507129+0j)
Xsfinite: (200.000000000000, 78.4323383366117, 200.000000000000, 200.000000000000)
X: [200.000000000000, 78.4323383366117, 200.000000000000, 200.000000000000]
l1= (-39.02831004251024+0j) ; l2= (53.9678209005901+0j) ; l3= (27.614358198819787+0j) ; l4= (-6.934323383366118+0j)
Xsfinite: (200.000000000000, 199.775628552713, 255.411125099920, 200.000000000000)
X: [200.000000000000, 199.775628552713, 255.411125099920, 200.000000000000]
l1= (88.85412346190526+0j) ; l2= (75.25550188848923+0j) ; l3= (20.521295816993472+0j) ; l4= (-8.701867536526336+0j)
Xsfinite: (200.000000000000, 199.799368980461, 200.000000000000, 200.000000000000)
X: [200.000000000000, 199.799368980461, 200.000000000000, 200.000000000000]
l1= (99.42113

In [ ]:
Xss #se muestran las soluciones estables

In [ ]:
Lss[0]  #se muestran los autovalores de dichas soluciones